In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.linalg
from sklearn.decomposition import PCA
import seaborn as sns; sns.set(style="white", color_codes=True)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [6]:
B1G_14_15_H = pd.read_csv('2014-15 B1G Data(hailey).csv')
print(B1G_14_15_H.columns)

B1G_18_19 = B1G_14_15_H[B1G_14_15_H['Year'] == '2018-2019']

Index(['School', 'Year', 'G', 'MP', 'FG-O', 'FG-T', 'FGA-O', 'FGA-T', 'FG%-O',
       'FG%-T', '2P-O', '2P-T', '2PA-O', '2PA-T', '2P%-O', '2P%-T', '3P-O',
       '3P-T', '3PA-O', '3PA-T', '3P%-O', '3P%-T', 'FT-O', 'FT-T', 'FTA-O',
       'FTA-T', 'FT%-O', 'FT%-T', 'ORB-O', 'ORB-T', 'DRB-O', 'DRB-T', 'TRB-O',
       'TRB-T', 'AST-O', 'AST-T', 'STL-O', 'STL-T', 'BLK-O', 'BLK-T', 'TOV-O',
       'TOV-T', 'PF-O', 'PF-T', 'PTS-O', 'PTS-T', 'PTS/G-O', 'PTS/G-T', 'W',
       'L', 'Seed', 'Finish'],
      dtype='object')


In [44]:
data = B1G_18_19.loc[56, :]
print(data)

B1G_18_19

School     Michigan State
Year            2018-2019
G                      20
MP                   4025
FG-O                  464
FG-T                  524
FGA-O                1233
FGA-T                1093
FG%-O               0.376
FG%-T               0.479
2P-O                  324
2P-T                  382
2PA-O                 774
2PA-T                 693
2P%-O               0.419
2P%-T               0.551
3P-O                  140
3P-T                  142
3PA-O                 459
3PA-T                 400
3P%-O               0.305
3P%-T               0.355
FT-O                  221
FT-T                  310
FTA-O                 326
FTA-T                 402
FT%-O               0.678
FT%-T               0.771
ORB-O                 251
ORB-T                 215
DRB-O                 398
DRB-T                 579
TRB-O                 649
TRB-T                 794
AST-O                 247
AST-T                 354
STL-O                 126
STL-T                 103
BLK-O       

,School,Year,G,MP,FG-O,FG-T,FGA-O,FGA-T,FG%-O,FG%-T,...,PF-O,PF-T,PTS-O,PTS-T,PTS/G-O,PTS/G-T,W,L,Seed,Finish
56,Michigan State,2018-2019,20,4025,464,524,1233,1093,0.376,0.479,...,356,328,1289,1500,64.5,75.0,0.80,0.20,1,1
57,Purdue,2018-2019,20,4050,445,508,1082,1173,0.411,0.433,...,342,354,1306,1465,65.3,73.3,0.80,0.20,2,8
58,Michigan,2018-2019,20,4000,464,503,1129,1147,0.411,0.439,...,285,295,1225,1370,61.3,68.5,0.75,0.25,3,2
59,Wisconsin,2018-2019,20,4100,440,492,1145,1123,0.384,0.438,...,343,315,1206,1305,60.3,65.3,0.70,0.30,4,4
60,Maryland,2018-2019,20,4000,456,469,1166,1078,0.391,0.435,...,333,311,1287,1341,64.4,67.1,0.65,0.35,5,12
61,Iowa,2018-2019,20,4050,546,512,1185,1173,0.461,0.436,...,400,344,1533,1501,76.7,75.1,0.50,0.50,6,8
62,Minnesota,2018-2019,20,4000,509,480,1154,1103,0.441,0.435,...,364,321,1399,1359,70.0,68.0,0.45,0.55,7,4
63,Ohio State,2018-2019,20,4025,471,454,1084,1106,0.435,0.410,...,338,369,1354,1291,67.7,64.6,0.40,0.60,8,8
64,Indiana,2018-2019,20,4100,507,475,1163,1137,0.436,0.418,...,358,354,1386,1320,69.3,66.0,0.40,0.60,9,12
65,Rutgers,2018-2019,20,4025,511,493,1160,1173,0.441,0.420,...,364,367,1455,1346,72.8,67.3,0.35,0.65,12,14


In [43]:
def score_calc(data):
    '''
    Calculates a team's effectiveness in a given year
    '''
    score_dict = dict() #intialize dictionary
    names = B1G_18_19["School"]
    
    #cycle through each B1G team
    for i in range(14):
        team = data.loc[i + 56, :] #isolate team's data
        
        #calculate factors and team score
        EGF = (team["FGA-T"] + 3*(team["3PA-T"])) / team["FGA-T"]
        TO = team["TOV-T"] / (team["G"] * 40)
        ORP = team["ORB-T"] / (team["ORB-T"] + team["DRB-O"])
        FTR = team["FT%-T"]
        
        team_score = .4*(EGF) + .25*(TO) + .2*(ORP) + .15*(FTR)
        
        name = names.loc[i+56]
        
        #store information in dictionary
        score_dict[name] = team_score

    return score_dict

score_dict = score_calc(B1G_18_19)
        
score_dict

{'Michigan State': 1.1030800988867313,
 'Purdue': 1.1552205929049686,
 'Michigan': 1.0863102362505506,
 'Wisconsin': 0.9958942150623906,
 'Maryland': 1.1085248899587936,
 'Iowa': 1.0877253726078138,
 'Minnesota': 0.9687699883721987,
 'Ohio State': 1.127665355710526,
 'Indiana': 1.0332540023950225,
 'Rutgers': 1.0013608593367502,
 'Penn State': 1.0652149582520982,
 'Illinois': 1.0627491872164367,
 'Nebraska': 1.0431438774947088,
 'Northwestern': 1.0849464346865574}

In [48]:
def predictor(team1, team2, score_dict):
    '''
    Returns the winner of two teams
    '''
    n = np.random.normal(scale = 1.5)
    if score_dict[team1] > score_dict[team2]:
        if n > score_dict[team1]:
            winner = team2
        else:
            winner = team1
    else:
        if n > score_dict[team2]:
            winner = team1
        else:
            winner = team2
            
    return winner

Michigan
Michigan State
Michigan State
Michigan
Michigan State
Michigan State
Michigan State
Michigan
Michigan State
Michigan State
Michigan
Michigan State
Michigan State
Michigan State
Michigan State
Michigan State
Michigan State
Michigan State
Michigan State
Michigan
